# CME 241 Assignment 13

## Shaan Patel

### Question 1

In [2]:
from typing import Iterable, Iterator, TypeVar, Mapping
import rl.markov_decision_process as mp
from rl.returns import returns
from collections import defaultdict

In [4]:
S = TypeVar('S')
A = TypeVar('A')

def tabular_mc_glie(
    traces: Iterable[Iterable[mp.TransitionStep[S,A]]],
    approx_0: Mapping[S,Mapping[A,float]],
    approx_pol: Mapping[S,Mapping[A,float]],
    gamma: float,
    episode_length_tolerance: float = 1e-6
) -> Iterator[Mapping[S,Mapping[A,float]]]:

    counts = defaultdict(lambda: 0)
    k = 0

    episodes: Iterator[Iterator[mp.ReturnStep[S]]] =\
        (returns(trace, gamma, episode_length_tolerance) for trace in traces)
    f = approx_0
    pi = approx_pol
    yield pi

    for episode in episodes:
        for s in episode:
            for a,r in episode[s]:
                counts[(s,a)] += 1
                f[s][a] += (1/counts[(s,a)])*(r - f[s][a])
        k += 1
        for s in f:
            maxpol = max(f[s], key = f[s].get)
            for a in f[s]:
                if a == maxpol:
                    pi[s][a] = (1/(k*len(f[s]))) + 1 - (1/k)
                else:
                    pi[s][a] = 1/(k*len(f[s]))
        yield pi


            

